In [2]:
#|default_exp app

#### Import required dependencies

In [4]:
#|export
from fastai.vision.all import *
import gradio as gr
from fastbook import *
from fastai.vision.widgets import ImageClassifierCleaner


#### Create dataset with bear images

- Identify categories
- Instantiate path
- Create directory 'bears' and search images for each category of bear
- Download

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
# if not path.exists():
path.mkdir(exist_ok=True)
for o in bear_types:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    results = search_images_ddg(f'{o} bear')
    download_images(dest, urls=results)

Saved image files in a variable

In [ ]:
fns = get_image_files(path)

 Remove images that fail


In [1]:
failed = verify_images(fns)
failed.map(Path.unlink);

NameError: name 'verify_images' is not defined

#### DataLoaders()

The `DataBlock` API helps us to fully customize every stage of the creation of `DataLoaders`. Let us go through the arguments:

1. `blocks` - argument specifying the data types for the independent (images) and dependent (category name) variables
2. `get_items` - specifies how to get a list of the files via file paths
3. `splitter` - instantiates RandomSplitter() class, assigns 20% of the data to validation set and sets seed
4. `get_y` - telling fastai what function to call to create labels in our dataset. In fastai, `parent_label` is a function that gets the name of the folder a file is in.
5. `item_tfms` - to group images in a tensor, they need to be of the same size. Item transforms resizes these images and they do them on each image.

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

#### Data Augmentation

Instead, what we normally do in practice is to randomly select part of the image, and crop to just that part. On each epoch (which is one complete pass through all of our images in the dataset) we randomly select a different part of each image. This means that our model can learn to focus on, and recognize, different features in our images. It also reflects how images work in the real world: different photos of the same thing may be framed in slightly different ways.

In fact, an entirely untrained neural network knows nothing whatsoever about how images behave. It doesn't even recognize that when an object is rotated by one degree, it still is a picture of the same thing! So actually training the neural network with examples of images where the objects are in slightly different places and slightly different sizes helps it to understand the basic concept of what an object is, and how it can be represented in an image.

Here's another example where we replace `Resize` with `RandomResizedCrop`, which is the transform that provides the behavior we just described. The most important parameter to pass in is `min_scale`, which determines how much of the image to select at minimum each time:

In [ ]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

#### Train Model

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Next, we also use ths trained model to clean up the train datasets

In [ ]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

### Load the model

In [ ]:
learn.export()

#### Loan trained model

In [5]:
#|export
learn = load_learner('export.pkl')

In [6]:
#|export

labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [ ]:
#|export
gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(512, 512)), \
    outputs=gr.outputs.Label(num_top_classes=3)).launch(share=True)

/var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/ipykernel_7999/1945180168.py:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(512, 512)), \
/var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/ipykernel_7999/1945180168.py:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(512, 512)), \
/var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/ipykernel_7999/1945180168.py:3: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Label(num_top_classes=3)).launch(share=True)
/var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/ipykernel_7999/1945180168.py:3: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, 

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://734bedcb57d7e8dc60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
